In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import evotsc

In [ ]:
# Population-level constants
genome_size = 10_000
gene_length = 100
intergene = 900
interaction_dist = 2500

In [ ]:
def generate_genes(nb_genes, gene_length, intergene):
    genes = []

    for gene in range(nb_genes):
        new_gene = evotsc.Gene(length=gene_length,
                               intergene=intergene,
                               orientation=np.random.randint(2),
                               basal_expression=np.random.random()) 
        genes.append(new_gene)

    return genes

In [ ]:
def plot_expr(temporal_expr, fitness, plot_name):
    nb_genes, nb_steps = temporal_expr.shape
    
    #colormap = mpl.cm.get_cmap('viridis', nb_genes)(range(nb_genes))
    
    plt.figure(figsize=(9, 6), dpi=200)
    
    for gene in range(nb_genes):
        linestyle = 'solid' if genes[gene].orientation == 0 else 'dashed'
        plt.plot(temporal_expr[gene, :],
                 linestyle=linestyle,
                 #color=colormap[gene],
                 label=f'Gene {gene}')
    
    plt.grid(linestyle=':')
    plt.xlabel('Time', fontsize='large')
    plt.ylabel('Expression level', fontsize='large')
    
    plt.legend(loc='center right')
    plt.title(f'Fitness: {fitness:.2}')
        
    plt.savefig(plot_name, dpi=300, bbox_inches='tight')

In [ ]:
genes = generate_genes(nb_genes=10, gene_length=gene_length, intergene=intergene)

In [ ]:
indiv = evotsc.Individual(genes=genes, interaction_dist=interaction_dist, nb_eval_steps=16)

In [ ]:
res, fitness = indiv.evaluate()

In [ ]:
plot_expr(res, fitness, 'baseline')

In [ ]:
def switch_1_gene(individual):
    for i_switched_gene in range(len(genes)):
        # Switch the current gene 
        to_switch = individual.genes[i_switched_gene]
        to_switch.orientation = 1 - to_switch.orientation

        # Run the switched system and plot
        switch_res, switch_fit = individual.evaluate()
        print(f'Switched gene {i_switched_gene}, fitness: {switch_fit}')
        plot_expr(switch_res, switch_fit, f'switched_{i_switched_gene}')

        # Reset the gene to its former orientation
        to_switch.orientation = 1 - to_switch.orientation

In [ ]:
switch_1_gene(indiv)